In [ ]:
import pandas as pd
import datacompy
import numpy as np
import os
from itertools import combinations

# from constant import COGNOS_TO_LOOKER

COGNOS_TO_LOOKER = {
    'Service Scope Group': 'Service Scope Group Code',
    'Dominant Leg Flag': 'Is Dominant (Yes / No)',
    'POR Location Code': 'POR Location Code',
    'SC/RFA Number': 'Contract Number',
    'SC/RFA Contract Type': 'Contract Type Name',
    'SC/RFA Group Customer Code': 'Contract Group Customer Code',
    'SC/RFA Group Customer Name': 'Contract Group Customer Name',
    'TEU Quantity':'Total TEU'
}

yes_no_map = {
    'N': 'No',
    'Y': 'Yes'
}

IS_DOMINANT_TEXT='Is Dominant (Yes / No)'

def create_diff_df(df_merged, columns):
    conditions = []
    for column in columns:
        data_cognos = df_merged[f"{column}_cognos"]
        data_looker = df_merged[f"{column}_looker"]
        if data_cognos.dtype== np.float64:
            data_cognos = data_cognos.round(2)
            data_looker = data_looker.round(2)
            condition = np.isclose(data_cognos, data_looker, atol=0.1)
        elif data_cognos.dtype== np.datetime64:
            rounded_cognos = data_cognos.dt.round('S')
            rounded_looker = data_looker.dt.round('S')
            condition = (rounded_cognos == rounded_looker)
        elif data_cognos.dtype== object:
            condition = (data_cognos.str.upper() == data_looker.str.upper())
        else:
            continue
        conditions.append(condition)
    combined_condition = np.all(conditions, axis=0)
    diff_df = df_merged[~combined_condition].copy()
    matching_df = df_merged[combined_condition].copy()
    diff_df = diff_df.applymap(lambda x: 'Not exists' if pd.isnull(x) else x)
    matching_df = matching_df.applymap(lambda x: 'Not exists' if pd.isnull(x) else x)
    return matching_df, diff_df

def filter_and_return_dataframes(df):
    df_missing_cognos = df.loc[df.filter(like='_cognos').eq('Not exists').all(axis=1)]
    df_missing_looker = df.loc[df.filter(like='_looker').eq('Not exists').all(axis=1)]
    df_remaining_diff = df.drop(df_missing_cognos.index).drop(df_missing_looker.index)
    return df_missing_cognos, df_missing_looker, df_remaining_diff.reset_index()

def split_dataframe(df):
    cols_df_cognos = [col for col in df.columns if not col.endswith("_looker")]
    df_cognos_new = df[cols_df_cognos]
    cols_df_looker = [col for col in df.columns if not col.endswith("_cognos")]
    df_looker_new = df[cols_df_looker]
    return df_cognos_new, df_looker_new

def compare_and_style_data(diff_df, keys, suffix_cognos, suffix_looker):
    def highlight_cells(val):
        if val == 'Same':
            return ''
        else:
            return 'background-color: red; color: white;'
    common_prefixes = set(col.split(suffix_cognos)[0] for col in diff_df.columns[2:])
    for prefix in common_prefixes:
        cognos_cols = [col for col in diff_df.columns if col.startswith(f'{prefix}{suffix_cognos}')]
        looker_cols = [col for col in diff_df.columns if col.startswith(f'{prefix}{suffix_looker}')]
        for cognos_col, looker_col in zip(cognos_cols, looker_cols):
            column_name = f'Compare_{prefix}_(Cognos <> Looker)'
            data_type = diff_df[cognos_col].dtype
            if data_type == np.float64:
                diff_df[cognos_col] = diff_df[cognos_col].round(2)
                diff_df[looker_col] = diff_df[looker_col].round(2)
                condition = np.isclose(diff_df[cognos_col], diff_df[looker_col], atol=0.1)
            elif data_type == np.datetime64:
                rounded_cognos = diff_df[cognos_col].dt.round('S')
                rounded_looker = diff_df[looker_col].dt.round('S')
                condition = (rounded_cognos == rounded_looker)
            elif data_type == object:
                condition = (diff_df[cognos_col].str.upper() == diff_df[looker_col].str.upper())
            else:
                continue
            diff_df[column_name] = diff_df.apply(lambda row: f'({row[cognos_col]} <> {row[looker_col]})'
                                                 if not condition[row.name]
                                                 else 'Same', axis=1)
    selected_keys = keys + [col for col in diff_df.columns if col.startswith('Compare')]
    styled_df = diff_df[selected_keys].style.applymap(highlight_cells, subset=pd.IndexSlice[:, diff_df[selected_keys].columns.str.startswith('Compare_')])
    return styled_df

COGNOS_PATH = r"C:\Users\hoa.nd\Desktop\WAP\compare\20230516_LA_Eagle-X_Account\Copy of 20230516_LA_Eagle-X_Account_cognos_full.xlsx"
LOOKER_PATH = r"C:\Users\hoa.nd\Desktop\WAP\compare\20230516_LA_Eagle-X_Account\Copy of 20230516_LA_Eagle-X_Account-looker_full.xlsx"

df_cognos = pd.read_excel(COGNOS_PATH, dtype=str)
colum_cognos_drops=['Trunk VVD Service Lane Code']
df_cognos=df_cognos.drop(colum_cognos_drops,axis=1)

df_looker = pd.read_excel(LOOKER_PATH,dtype=str)

# colum_looker_drops=['CM applicable','COA Trunk Service Code']
colum_looker_drops=['COA Trunk Service Code']

df_looker=df_looker.drop(colum_looker_drops,axis=1)

df_cognos=df_cognos.drop(['CM applicable'],axis=1)
df_cognos.rename(columns=COGNOS_TO_LOOKER, inplace=True)
df_cognos[IS_DOMINANT_TEXT] = df_cognos[IS_DOMINANT_TEXT].map(yes_no_map)

# ['Sales Month', 'Sales Week', 'Service Scope Group Code',
#        'Service Scope Code', 'Revenue Lane Code', 'Is Dominant (Yes / No)',
#        'POR Location Code', 'Trunk POL Location Code',
#        'Trunk POD Location Code', 'DEL Location Code', 'Contract Number',
#        'Contract Type Name', 'Contract Group Customer Code',
#        'Contract Group Customer Name', 'Total TEU']
not_keys= ['Trunk POL Location Code','Trunk POD Location Code','Total TEU','Contract Type Name']

keys_list = [x for x in list(df_cognos.columns) if x not in not_keys]



# combinations_2 = list(combinations(keys_list, len(keys_list)-3))
# for combo in combinations_2:
#     key=list(combo)
#     compare = datacompy.Compare( df_cognos, df_looker, join_columns=key, df1_name='Cognos', df2_name='Looker')
#     print(key,compare.report())



# compare = datacompy.Compare(
# df_cognos, df_looker, 
# join_columns=keys_list, df1_name='Cognos', df2_name='Looker')
# print(compare.report())

df_merged = pd.merge(df_cognos, df_looker, on=keys_list, how='inner', suffixes=('_cognos', '_looker'))
matching_df, diff_df = create_diff_df(df_merged, not_keys)
# matching_df.to_excel(os.path.join(r'C:\Users\hoa.nd\Desktop\WAP\compare\20230516_LA_Eagle-X_Account\result', 'Match.xlsx'), index=False)
df_missing_cognos, df_missing_looker, df_remaining_diff = filter_and_return_dataframes(diff_df)

df_cognos_diff_new, df_looker_diff_new = split_dataframe(df_remaining_diff)
df_not_in_cognos, df_only_in_looker = split_dataframe(df_missing_cognos)
df_only_in_cognos, df_not_in_looker = split_dataframe(df_missing_looker)


different_df = df_remaining_diff
selected_keys = keys_list
styled_result = compare_and_style_data(different_df, selected_keys, '_cognos', '_looker')
# styled_result.to_excel(os.path.join(r'C:\Users\hoa.nd\Desktop\WAP\compare\20230516_LA_Eagle-X_Account\result', 'Compare_differences.xlsx'), index=False)
styled_result    


